# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 2 2022 3:54PM,245675,16,SAR00000367,Sartorius Stedim North America,Released
1,Sep 2 2022 3:47PM,245673,16,SAR00000366,Sartorius Stedim North America,Released
2,Sep 2 2022 3:43PM,245671,18,TASA-272064,TASA USA Inc.,Released
3,Sep 2 2022 3:39PM,245670,18,TASA-272063,TASA USA Inc.,Released
4,Sep 2 2022 3:34PM,245669,16,TASA-280060,TASA USA Inc.,Released
5,Sep 2 2022 3:33PM,245667,16,SHL-8476270,"SHL Pharma, LLC",Executing
6,Sep 2 2022 3:31PM,245666,16,TASA-277119,TASA USA Inc.,Released
7,Sep 2 2022 2:52PM,245663,10,EYE082922EM.1,Eye Pharma Inc,Cancelled
8,Sep 2 2022 2:32PM,245658,20,8475644,"Exact-Rx, Inc.",Released
9,Sep 2 2022 2:32PM,245658,20,8475645,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,245669,Released,1
22,245670,Released,1
23,245671,Released,1
24,245673,Released,1
25,245675,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245669,NaN,NaN,NaN,1.0
245670,NaN,NaN,NaN,1.0
245671,NaN,NaN,NaN,1.0
245673,NaN,NaN,NaN,1.0
245675,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245606,0.0,0.0,0.0,1.0
245609,0.0,0.0,0.0,1.0
245611,0.0,0.0,0.0,1.0
245613,0.0,0.0,1.0,0.0
245619,0.0,0.0,0.0,22.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245606,0.0,0,0,1
245609,0.0,0,0,1
245611,0.0,0,0,1
245613,0.0,0,1,0
245619,0.0,0,0,22


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245606,0.0,0,0,1
1,245609,0.0,0,0,1
2,245611,0.0,0,0,1
3,245613,0.0,0,1,0
4,245619,0.0,0,0,22


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245606,0.0,,,1
1,245609,0.0,,,1
2,245611,0.0,,,1
3,245613,0.0,,1,
4,245619,0.0,,,22


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 2 2022 3:54PM,245675,16,Sartorius Stedim North America
1,Sep 2 2022 3:47PM,245673,16,Sartorius Stedim North America
2,Sep 2 2022 3:43PM,245671,18,TASA USA Inc.
3,Sep 2 2022 3:39PM,245670,18,TASA USA Inc.
4,Sep 2 2022 3:34PM,245669,16,TASA USA Inc.
5,Sep 2 2022 3:33PM,245667,16,"SHL Pharma, LLC"
6,Sep 2 2022 3:31PM,245666,16,TASA USA Inc.
7,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc
8,Sep 2 2022 2:32PM,245658,20,"Exact-Rx, Inc."
31,Sep 2 2022 2:17PM,245653,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Sep 2 2022 3:54PM,245675,16,Sartorius Stedim North America,0.0,,,1
1,Sep 2 2022 3:47PM,245673,16,Sartorius Stedim North America,0.0,,,1
2,Sep 2 2022 3:43PM,245671,18,TASA USA Inc.,0.0,,,1
3,Sep 2 2022 3:39PM,245670,18,TASA USA Inc.,0.0,,,1
4,Sep 2 2022 3:34PM,245669,16,TASA USA Inc.,0.0,,,1
5,Sep 2 2022 3:33PM,245667,16,"SHL Pharma, LLC",0.0,,1,
6,Sep 2 2022 3:31PM,245666,16,TASA USA Inc.,0.0,,,1
7,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc,1.0,,,
8,Sep 2 2022 2:32PM,245658,20,"Exact-Rx, Inc.",0.0,,,23
9,Sep 2 2022 2:17PM,245653,10,"Methapharm, Inc.",0.0,,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 3:54PM,245675,16,Sartorius Stedim North America,1,,
1,Sep 2 2022 3:47PM,245673,16,Sartorius Stedim North America,1,,
2,Sep 2 2022 3:43PM,245671,18,TASA USA Inc.,1,,
3,Sep 2 2022 3:39PM,245670,18,TASA USA Inc.,1,,
4,Sep 2 2022 3:34PM,245669,16,TASA USA Inc.,1,,
5,Sep 2 2022 3:33PM,245667,16,"SHL Pharma, LLC",,1,
6,Sep 2 2022 3:31PM,245666,16,TASA USA Inc.,1,,
7,Sep 2 2022 2:52PM,245663,10,Eye Pharma Inc,,,
8,Sep 2 2022 2:32PM,245658,20,"Exact-Rx, Inc.",23,,
9,Sep 2 2022 2:17PM,245653,10,"Methapharm, Inc.",9,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 3:54PM,245675,16,Sartorius Stedim North America,1,,
1,Sep 2 2022 3:47PM,245673,16,Sartorius Stedim North America,1,,
2,Sep 2 2022 3:43PM,245671,18,TASA USA Inc.,1,,
3,Sep 2 2022 3:39PM,245670,18,TASA USA Inc.,1,,
4,Sep 2 2022 3:34PM,245669,16,TASA USA Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 3:54PM,245675,16,Sartorius Stedim North America,1.0,NaN,NaN
1,Sep 2 2022 3:47PM,245673,16,Sartorius Stedim North America,1.0,NaN,NaN
2,Sep 2 2022 3:43PM,245671,18,TASA USA Inc.,1.0,NaN,NaN
3,Sep 2 2022 3:39PM,245670,18,TASA USA Inc.,1.0,NaN,NaN
4,Sep 2 2022 3:34PM,245669,16,TASA USA Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 3:54PM,245675,16,Sartorius Stedim North America,1.0,0.0,0.0
1,Sep 2 2022 3:47PM,245673,16,Sartorius Stedim North America,1.0,0.0,0.0
2,Sep 2 2022 3:43PM,245671,18,TASA USA Inc.,1.0,0.0,0.0
3,Sep 2 2022 3:39PM,245670,18,TASA USA Inc.,1.0,0.0,0.0
4,Sep 2 2022 3:34PM,245669,16,TASA USA Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,736935,31.0,0.0,0.0
12,491271,3.0,1.0,0.0
16,2947685,10.0,2.0,0.0
18,736965,3.0,0.0,0.0
20,736910,39.0,0.0,1.0
21,245646,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,736935,31.0,0.0,0.0
1,12,491271,3.0,1.0,0.0
2,16,2947685,10.0,2.0,0.0
3,18,736965,3.0,0.0,0.0
4,20,736910,39.0,0.0,1.0
5,21,245646,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,0.0,0.0
1,12,3.0,1.0,0.0
2,16,10.0,2.0,0.0
3,18,3.0,0.0,0.0
4,20,39.0,0.0,1.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,12,Released,3.0
2,16,Released,10.0
3,18,Released,3.0
4,20,Released,39.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0
Executing,0.0,1.0,2.0,0.0,0.0,1.0
Released,31.0,3.0,10.0,3.0,39.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,20,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,0.0,1.0,2.0,0.0,0.0,1.0
2,Released,31.0,3.0,10.0,3.0,39.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,20,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,0.0,1.0,2.0,0.0,0.0,1.0
2,Released,31.0,3.0,10.0,3.0,39.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()